## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
# 參考 https://keras.io/zh/applications/
#model=keras.applications.xception.Xception(include_top=False, weights='imagenet',input_tensor=input_tensor, pooling=None, classes=10)

'''Resnet 50 架構'''
model=keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor, pooling=None, classes=10)
model.summary()

C:\Users\fired\.conda\envs\env_py35\lib\site-packages\keras_applications\resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

## 添加層數

In [3]:
x = model.output

'''可以參考Cifar10實作章節'''
x = GlobalAveragePooling2D()(x)
x = Dense(output_dim=128, activation='relu')(x)
x = Dropout(p=0.1)(x)

predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 178


C:\Users\fired\.conda\envs\env_py35\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=128, activation="relu")`
  """
C:\Users\fired\.conda\envs\env_py35\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
C:\Users\fired\.conda\envs\env_py35\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, activation="softmax")`
  


## 鎖定特定幾層不要更新權重

In [4]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


C:\Users\fired\.conda\envs\env_py35\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 89s 2ms/step - loss: 1.5677 - acc: 0.5658
Epoch 2/100
50000/50000 [==============================] - 81s 2ms/step - loss: 0.9640 - acc: 0.6687
Epoch 3/100
50000/50000 [==============================] - 79s 2ms/step - loss: 0.8383 - acc: 0.7116
Epoch 4/100
50000/50000 [==============================] - 79s 2ms/step - loss: 0.7740 - acc: 0.7280
Epoch 5/100
50000/50000 [==============================] - 82s 2ms/step - loss: 0.7095 - acc: 0.7548
Epoch 6/100
50000/50000 [==============================] - 80s 2ms/step - loss: 0.6487 - acc: 0.7782
Epoch 7/100
50000/50000 [==============================] - 82s 2ms/step - loss: 0.5866 - acc: 0.7997
Epoch 8/100
50000/50000 [==============================] - 80s 2ms/step - loss: 0.4766 - acc: 0.8365
Epoch 9/100
50000/50000 [==============================] - 80s 2ms/step - loss: 0.4491 - acc: 0.8443
Epoch 10/100
50000/50000 [==============================] - 80s 2ms/step - loss: 0.5252 - a

50000/50000 [==============================] - 83s 2ms/step - loss: 0.1148 - acc: 0.9602
Epoch 82/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.1157 - acc: 0.9631
Epoch 83/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.1165 - acc: 0.9646
Epoch 84/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.1147 - acc: 0.9648
Epoch 85/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.1752 - acc: 0.9558
Epoch 86/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.1361 - acc: 0.9540
Epoch 87/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.1131 - acc: 0.9627
Epoch 88/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.1010 - acc: 0.9658
Epoch 89/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.1001 - acc: 0.9652
Epoch 90/100
50000/50000 [==============================] - 83s 2ms/step - loss: 0.1033 - acc: 